In [1]:
import tensorflow
import pandas as pd
import time
import numpy as np
import math


# tensorflow libraries
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, concatenate, Input, Flatten

from tensorflow.keras import layers
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup
from tensorflow.keras import optimizers

# sklearn libraries are useful for preprocessing, performance measures, etc.
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

Num GPUs Available:  1


In [2]:
import pandas as pd
df = pd.read_csv('file:/root/bryan/NG/csv-files/training.csv')
val_df = pd.read_csv('file:/root/bryan/NG/csv-files/validation.csv')

lat = df.pop("lat_3414")
lon = df.pop('lon_3414')
df.pop("floor_id")
df.pop("timestamp")
df.pop("type")
df.pop("latitude")
df.pop("longitude")


val_lat = val_df.pop("lat_3414")
val_lon = val_df.pop('lon_3414')
val_df.pop("floor_id")
val_df.pop("timestamp")
val_df.pop("type")
val_df.pop("latitude")
val_df.pop("longitude")


0      103.851602
1      103.851598
2      103.851594
3      103.851590
4      103.851586
          ...    
692    103.851100
693    103.851105
694    103.851110
695    103.851114
696    103.851119
Name: longitude, Length: 697, dtype: float64

In [3]:
from math import radians, cos, sin, asin, sqrt

def single_pt_haversine(lat, lng, degrees=True):
    """
    'Single-point' Haversine: Calculates the great circle distance
    between a point on Earth and the (0, 0) lat-long coordinate
    """
    r = 6371 # Earth's radius (km). Have r = 3956 if you want miles

    # Convert decimal degrees to radians
    if degrees:
        lat, lng = map(radians, [lat, lng])

    # 'Single-point' Haversine formula
    a = sin(lat/2)**2 + cos(lat) * sin(lng/2)**2
    d = 2 * r * asin(sqrt(a)) 

    return d
    

In [4]:
def dataframe_to_dataset(dataframe, label):
    dataframe = dataframe.copy()
    labels = label
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds

In [5]:
labels = []
for i in range (len(lat)):
  labels.append(single_pt_haversine(lat[i],lon[i]))
  # temp = []
  # temp.append(lat[i])
  # temp.append(lon[i])
  # labels.append(temp)


val_labels = []
for i in range (len(val_lat)):
  val_labels.append(single_pt_haversine(val_lat[i],val_lon[i]))
  # temp = []
  # temp.append(val_lat[i])
  # temp.append(val_lon[i])
  # val_labels.append(temp)

print(val_labels[2])


#print(val_df.iloc[[2]])

10067.921913690923


In [6]:
train_ds = dataframe_to_dataset(df, labels)

In [7]:

# report which features were selected by RFE
from sklearn.feature_selection import RFE
# define dataset
X, y = df, labels
# define RFE
rfe = RFE(estimator=RandomForestRegressor(), n_features_to_select=400)
# fit RFE
rfe.fit(X, y)
# summarize all features
for i in range(X.shape[1]):
	print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))

Column: 0, Selected False, Rank: 447.000
Column: 1, Selected False, Rank: 289.000
Column: 2, Selected False, Rank: 369.000
Column: 3, Selected False, Rank: 160.000
Column: 4, Selected False, Rank: 368.000
Column: 5, Selected False, Rank: 180.000
Column: 6, Selected False, Rank: 112.000
Column: 7, Selected False, Rank: 350.000
Column: 8, Selected False, Rank: 296.000
Column: 9, Selected False, Rank: 314.000
Column: 10, Selected True, Rank: 1.000
Column: 11, Selected True, Rank: 1.000
Column: 12, Selected True, Rank: 1.000
Column: 13, Selected False, Rank: 158.000
Column: 14, Selected False, Rank: 363.000
Column: 15, Selected False, Rank: 303.000
Column: 16, Selected False, Rank: 290.000
Column: 17, Selected False, Rank: 398.000
Column: 18, Selected False, Rank: 172.000
Column: 19, Selected False, Rank: 185.000
Column: 20, Selected False, Rank: 391.000
Column: 21, Selected True, Rank: 1.000
Column: 22, Selected False, Rank: 346.000
Column: 23, Selected False, Rank: 354.000
Column: 24, Se

In [8]:
def get_index_positions(list_of_elems, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements '''
    index_pos_list = []
    index_pos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            index_pos = list_of_elems.index(element, index_pos)
            # Add the index position in list
            index_pos_list.append(index_pos)
            index_pos += 1
        except ValueError as e:
            break
    return index_pos_list

In [9]:
selected = [] 
for i in range(X.shape[1]):
	#print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
  selected.append(rfe.support_[i])

print((selected[52]))

index_pos_list = get_index_positions(selected, False)
print((index_pos_list))



False
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 25, 26, 28, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 78, 79, 80, 87, 88, 89, 90, 91, 92, 93, 94, 95, 102, 103, 104, 108, 109, 110, 117, 118, 120, 122, 123, 126, 128, 129, 130, 131, 132, 133, 134, 135, 142, 143, 144, 145, 146, 147, 148, 155, 156, 157, 158, 159, 160, 161, 162, 169, 170, 171, 172, 180, 181, 186, 191, 192, 193, 198, 199, 200, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 239, 240, 241, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 273, 274, 275, 276, 277, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 307, 308, 309, 310, 312, 317, 318, 319, 320, 321, 322, 329, 330, 331, 332, 333, 334, 335, 337, 346, 357, 358, 359, 360, 361, 362, 363, 364, 366, 367, 368, 381, 382, 383, 384, 385, 392, 393, 398, 399, 400, 408, 409, 413, 414, 415, 416, 417, 419, 420, 421, 425,

In [10]:
df_tosave = pd.read_csv('/root/bryan/NG/csv-files/training.csv')
val_df_tosave = pd.read_csv('/root/bryan/NG/csv-files/validation.csv')

In [11]:
df_tosave.drop(df_tosave.columns[index_pos_list], axis = 1, inplace = True) 
 

In [12]:
df_tosave.head()

,WAP10,WAP11,WAP12,WAP21,WAP24,WAP27,WAP29,WAP30,WAP31,WAP32,...,WAP838,WAP844,WAP846,floor_id,timestamp,longitude,latitude,type,lon_3414,lat_3414
0,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-110,-110,2,1.630000e+12,103.851602,1.290576,truth,30034.832791,30330.891589
1,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-84,-110,2,1.630000e+12,103.851598,1.290569,pseudo,30034.365372,30330.111037
2,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-110,-110,2,1.630000e+12,103.851594,1.290562,pseudo,30033.897954,30329.330486
3,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-87,-110,2,1.630000e+12,103.851590,1.290555,pseudo,30033.419406,30328.549935
4,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-80,-110,2,1.630000e+12,103.851585,1.290548,pseudo,30032.951988,30327.769384


In [13]:
df_tosave.to_csv("/root/bryan/NG/csv-files/rfe-train.csv",index=False)

In [14]:
val_df_tosave.drop(val_df_tosave.columns[index_pos_list], axis = 1, inplace = True)
val_df_tosave.head()


,WAP10,WAP11,WAP12,WAP21,WAP24,WAP27,WAP29,WAP30,WAP31,WAP32,...,WAP838,WAP844,WAP846,floor_id,timestamp,longitude,latitude,type,lon_3414,lat_3414
0,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-110,-110,2,1.630000e+12,103.851602,1.290576,truth,30034.832791,30330.891589
1,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-91,-110,2,1.630000e+12,103.851598,1.290569,pseudo,30034.387630,30330.143547
2,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-110,-110,2,1.630000e+12,103.851594,1.290563,pseudo,30033.931341,30329.395505
3,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-110,-110,2,1.630000e+12,103.851590,1.290556,pseudo,30033.486180,30328.647463
4,-110,-110,-110,-110,-110,-110,-110,-110,-110,-110,...,-110,-110,-110,2,1.630000e+12,103.851586,1.290549,pseudo,30033.029891,30327.899420


In [15]:
val_df_tosave.to_csv("/root/bryan/NG/csv-files/rfe-test.csv",index=False)